[Reference](https://towardsdatascience.com/data-driven-marketing-attribution-1a28d2e613a0)

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

def subsets(S):
    '''Returns all possible subsets of the given set'''
    s = []
    for i in range(1, len(S)+1):
        s.extend(map(list, combinations(S, i)))
    return list(map('+'.join, s))

N = sorted({'Facebook', 'Google', 'LinkedIn'})

coalitions = subsets(N)
coalitions_lbl = ['S{}'.format(i) for i in range(1, len(coalitions)+1)]

# The coalition 'Facebook+Google' (S4) resulted in 10 conversions from 100 opportunities,
# so has a conversion ratio of 10/100 = 0.1 (10%)
IR = np.array([0.18, 0.04, 0.08, 0.1, 0.26, 0.07, 0.27])

pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame({
    'Coalition': coalitions,
    'Ratio': IR
}, dtype=np.float64, index=coalitions_lbl)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised


,Coalition,Ratio
S1,Facebook,0.18
S2,Google,0.04
S3,LinkedIn,0.08
S4,Facebook+Google,0.10
S5,Facebook+LinkedIn,0.26
S6,Google+LinkedIn,0.07
S7,Facebook+Google+LinkedIn,0.27


In [2]:
# B is binary squared matrix that represents coalition membership.
# For example, coalition Facebook+LinkedIn (S5) includes members:
# - Facebook (S1)
# - LinkedIn (S3)
# - Facebook+LinkedIn (S5)
# Resulting in the coefficients: [1,0,1,0,1,0,0]

d = 2**len(N)-1
B = np.matrix(np.zeros((d, d)))

for i in range(0, d):
    A = coalitions[i]
    S = subsets(A.split('+'))
    coef = [1 if c in S else 0 for c in coalitions]
    B[i] = coef

pd.options.display.float_format = '{:,.0f}'.format
pd.DataFrame(data=B, index=coalitions, columns=coalitions)


,Facebook,Google,LinkedIn,Facebook+Google,Facebook+LinkedIn,Google+LinkedIn,Facebook+Google+LinkedIn
Facebook,1,0,0,0,0,0,0
Google,0,1,0,0,0,0,0
LinkedIn,0,0,1,0,0,0,0
Facebook+Google,1,1,0,1,0,0,0
Facebook+LinkedIn,1,0,1,0,1,0,0
Google+LinkedIn,0,1,1,0,0,1,0
Facebook+Google+LinkedIn,1,1,1,1,1,1,1


In [3]:
# The product of the matrices coalition membership and coalition ratios
# is the coalition worth - the result of the characteristic function 'v(S)'

vS = np.dot(B, IR)
vS = np.squeeze(np.asarray(vS))

vSx = ['v({})'.format(lbl) for lbl in coalitions_lbl]
pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame({
    'Coalition': coalitions,
    'Worth': vS
}, index=vSx)

,Coalition,Worth
v(S1),Facebook,0.18
v(S2),Google,0.04
v(S3),LinkedIn,0.08
v(S4),Facebook+Google,0.32
v(S5),Facebook+LinkedIn,0.52
v(S6),Google+LinkedIn,0.19
v(S7),Facebook+Google+LinkedIn,1.00


In [4]:
from collections import defaultdict
from math import factorial

# Calculate the Shapley values - the average value of each channel's marginal contribution
# to the grand coalition, taking into account all possible orderings.

shapley = defaultdict(int)
n = len(N)

for i in N:
    for A in coalitions:
        S = A.split('+')
        if i not in S:
            k = len(S) # Cardinality of set |S|
            Si = S
            Si.append(i)
            Si = '+'.join(sorted(Si))
            # Weight = |S|!(n-|S|-1)!/n!
            weight = (factorial(k) * factorial(n-k-1)) / factorial(n)
            # Marginal contribution = v(S U {i})-v(S)
            contrib = vS[coalitions.index(Si)] - vS[coalitions.index(A)]            
            shapley[i] += weight * contrib
    shapley[i] += vS[coalitions.index(i)]/n

In [5]:
pd.options.display.float_format = '{:,.3f}'.format
pd.DataFrame({
    'Shapley value': list(shapley.values())
}, index=list(shapley.keys()))

,Shapley value
Facebook,0.450
Google,0.215
LinkedIn,0.335
